**Introdução Algoritmos Genéticos**

*   Exemplo de codificação binária;
*   Calculo binário para função f(x) = x²;
*   Seleção por torneio;

**Atividade para a próxima semana:**
*   Desenvolver o procedimento iterativo;
*   Função de cruzamento;
*   Função de mutação.

In [1]:
# genetic algorithm search for continuous function optimization
from numpy.random import randint
from numpy.random import rand

In [2]:
# objective function
def objective(x):
  # convert string to integer (decimal base)
  integer = int(x, 2)            
  return integer	

In [3]:
# decode bitstring to numbers
def decode(n_bits, bitstring):
      decoded = list()    
      # convert bitstring to a string of chars
      chars = ''.join([str(s) for s in bitstring])      
      return chars

In [4]:
# tournament selection
def tournament(pop, scores, k):
    # first random selection
    selection_ix = randint(len(pop))
    # Generate a randon list with k-1 individuals
    for ix in randint(0, len(pop), k - 1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]


def torneio(amostra, scores, maxQtd):
    while len(amostra) > maxQtd:
        while True:
            i1, i2 = randint(0, len(amostra), 2)
            if i1 != i2:
                break
        somaI1 = sum(amostra[i1])
        somaI2 = sum(amostra[i2])
        if somaI1 < somaI2:
            amostra.pop(i1)
            scores.pop(i1)
        elif somaI1 == somaI2:
            if scores[i1] < scores[i2]:
                amostra.pop(i1)
                scores.pop(i1)
            else:
                amostra.pop(i2)
                scores.pop(i2)
        else:
            amostra.pop(i2)
            scores.pop(i2)

In [5]:

# genetic algorithm
def genetic_algorithm(objective, n_bits, n_pop, k):
	# initial population of random bitstring
	pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
	print("Populacao inicial:")
	print(pop)
	for i in pop:
		print(i)
	# keep track of best solution
	best, best_eval = 0, objective(decode(n_bits, pop[0]))
	# decode population
	decoded = [decode(n_bits, p) for p in pop]    
	# evaluate all candidates in the population
	scores = [objective(d) for d in decoded]
	# select parents (mating pool)
	selected = [tournament(pop, scores, k) for _ in range(n_pop)]		
	return [scores, selected]

In [6]:
# bits per variable
n_bits = 5
# define the population size
n_pop = 5
# tornament selection k
k = 3
# perform the genetic algorithm search
fitness,tourSelec = genetic_algorithm(objective,n_bits, n_pop, k)
selectFit = []
for i in tourSelec:
  selectFit.append(objective(decode(n_bits,i)))

print("Resultado apos a selecao:")
for x, y, z in zip(fitness, tourSelec, selectFit):  
  print(x,y,z)

Populacao inicial:
[[1, 1, 0, 1, 0], [1, 1, 1, 1, 0], [0, 0, 0, 0, 1], [1, 1, 0, 0, 1], [0, 0, 1, 0, 1]]
[1, 1, 0, 1, 0]
[1, 1, 1, 1, 0]
[0, 0, 0, 0, 1]
[1, 1, 0, 0, 1]
[0, 0, 1, 0, 1]
Resultado apos a selecao:
26 [0, 0, 1, 0, 1] 5
30 [0, 0, 0, 0, 1] 1
1 [0, 0, 1, 0, 1] 5
25 [0, 0, 0, 0, 1] 1
5 [0, 0, 0, 0, 1] 1


In [7]:
geracao = 0
mut = 0


def crossOver(individuo_1, individuo_2):
    ponto_de_corte = randint(1, len(individuo_1) - 1)
    filho_1 = individuo_1[:ponto_de_corte] + individuo_2[ponto_de_corte:]
    filho_2 = individuo_2[:ponto_de_corte] + individuo_1[ponto_de_corte:]
    return filho_1, filho_2


def mutacao(individuo, probabilidade_de_mutacao):
    global mut
    stts = False
    for i in range(len(individuo)):
        if rand() <= probabilidade_de_mutacao:
            stts = True
            individuo[i] = 1 if individuo[i] == 0 else 0
    if stts:
        mut += 1
    return individuo


def iteracao(amostra, probabilidade_de_mutacao):
    nova_amostra = []
    for i in range(len(amostra)):
        filho_1, filho_2 = crossOver(amostra[i], amostra[(i - len(amostra) + 1)])
        nova_amostra.append(filho_1)
        nova_amostra.append(filho_2)
    for i in nova_amostra:
        i = mutacao(i, probabilidade_de_mutacao)
    return nova_amostra


def recursao(amostra, probabilidade_de_mutacao=0.01, iteracoes=-1):
    global geracao
    geracao += 1
    amostra = iteracao(amostra, probabilidade_de_mutacao)
    fit = []
    for i in amostra:
        fit.append(objective(decode(n_bits, i)))
    torneio(amostra, fit, 5)
    # print("Resultado apos a selecao:")
    # for y, z in zip(amostra, fit):
    #     print(y, z)
    if max(fit) != 31 and (iteracoes == -1 or iteracoes > 0):
        del fit
        recursao(amostra)
    else:
        print("Geração final: ")
        for y, z in zip(amostra, fit):
            print(y, z)


In [8]:

print("Geranção 0:")
for y, z in zip(tourSelec, selectFit):
        print(y, z)
recursao(tourSelec)
print ("Geracao: ", geracao)
print ("Mutacao: ", mut)

Geranção 0:
[0, 0, 1, 0, 1] 5
[0, 0, 0, 0, 1] 1
[0, 0, 1, 0, 1] 5
[0, 0, 0, 0, 1] 1
[0, 0, 0, 0, 1] 1
Geração final: 
[0, 1, 1, 0, 1] 13
[1, 1, 1, 0, 1] 29
[1, 1, 1, 0, 1] 29
[1, 1, 1, 1, 1] 31
[0, 1, 1, 0, 1] 13
Geracao:  17
Mutacao:  6
